# Classification of roads

We're going to look into the success of classifying algorithms on the LiDAR data we have with a couple of different techniques

### The following are the list of classifications defined by IFP
- 0 not yet classified (nothing done yet)
- 1 unclassified (actively marked as nothing)
- 2 ground, sidewalk
- 3,4,5 vegetation, low(gras) medium(shrubbery) high (trees)
- 6 buildings
- 8 street furniture
- 10 street markings
- 11 street, pavement
- 12 bike lanes
- 13 temporary things(bicycles, trashcans)
- 15 cars, trucks

In [ ]:
import laspy
import numpy as np
import matplotlib.pyplot as plt     
import seaborn as sns
import open3d as o3d
import pandas as pd
from dotenv import load_dotenv
import json
from upath import UPath
import os
# Load environment variables from .env file if it exists
load_dotenv()
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from src import data_loader

sns.set_theme(style="whitegrid")

In [ ]:
data_loader.fetch_and_process_lidar("riga.laz")

In [ ]:
import sys

def las_in_memory_size(las):
    """
    Estimate the in-memory size (in bytes) of a laspy LAS object.
    """
    total_size = 0
    for dim in list(las.point_format.dimension_names):
        arr = getattr(las, dim)
        total_size += arr.nbytes if hasattr(arr, 'nbytes') else sys.getsizeof(arr)
    return total_size

# Example usage:
# las = laspy.read("your_file.las")
# print(f"In-memory size: {las_in_memory_size(las)/1e6:.2f} MB")

def describe_las(las):
    print(f"In-memory size: {las_in_memory_size(las)/1e6:.2f} MB")
    print(f"Point Format: {las.header.point_format}")
    print(f"Number of Points: {las.header.point_count}")
    print("Available Dimensions:", list(las.point_format.dimension_names))
    print("Bounding Box:")
    print(f"  X: {las.header.mins[0]} to {las.header.maxs[0]}")
    print(f"  Y: {las.header.mins[1]} to {las.header.maxs[1]}")
    print(f"  Z: {las.header.mins[2]} to {las.header.maxs[2]}")
    print("Scale:", las.header.scales)
    print("Offset:", las.header.offsets)
    try:
        print("CRS:", las.header.parse_crs())
    except:
        print("CRS: Not defined")

In [ ]:
las = laspy.read("../data/bologna_filtered.las")

In [ ]:
describe_las(las)

### The las is pretty huge, so we need to sample it up (ensemble)

In [ ]:
from laspy import read
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Fields of interest
fields = ['X', 'Y', 'Z', 'intensity', 'red', 'green', 'blue', 'classification']

df = pd.DataFrame({field: np.asarray(getattr(las, field)) for field in fields})

# Optimize data types to reduce memory consumption by ~30%
df['intensity'] = df['intensity'].astype(np.uint16)
df['red'] = df['red'].astype(np.uint16)
df['green'] = df['green'].astype(np.uint16)
df['blue'] = df['blue'].astype(np.uint16)
df['classification'] = df['classification'].astype(np.uint8)  # Only need 0-15 values

print(f"✅ Data loaded with optimized dtypes")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")


In [ ]:
def make_sample(df, n=2_000_000, seed=None):
    return df.sample(n=n, random_state=seed).reset_index(drop=True)

# NOTE: Don't store all samples in memory at once
# Instead, create them on-demand during training to save ~8GB of RAM
print("✅ Sample function ready. Samples will be created on-demand.")


## Supervised 

In [ ]:
# -------------------------
# 0. Imports
# -------------------------
import pandas as pd
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
random_state = 42
#import tensorflow as tf
#from tensorflow.keras import layers, models
#import joblib

## Experiment A: Using all of the pointcloud classes


In [ ]:
def train_and_eval(X_train, X_test, y_train, y_test, class_weight="balanced", n_estimators=200):
    clf = RandomForestClassifier(n_estimators=n_estimators,
                                 class_weight=class_weight,
                                 n_jobs=-1,
                                 random_state=random_state)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    report = classification_report(y_test, y_pred, digits=3, output_dict=True)
    cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
    return clf, clf.classes_, report, cm

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns

def plot_confusion(cm, classes, title="Confusion matrix", normalize=False, cmap="Blues"):
    """
    Plots a confusion matrix, with optional normalization to percentages.

    Args:
        cm (array): confusion matrix (from sklearn.metrics.confusion_matrix)
        classes (list): list of class names
        title (str): plot title
        normalize (bool): if True, show percentages instead of counts
        cmap (str): color map
    """

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        fmt = ".2f"
        title = title + " (normalized %)"
    else:
        fmt = "d"

    plt.figure(figsize=(7, 6))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap=cmap, 
                xticklabels=classes, yticklabels=classes,
                cbar=True, square=True)

    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import gc

# --- Config ---
features = ['Z', 'intensity', 'red', 'green', 'blue'] # take away X and Y as they are likely to differ spatially if applied to different cities.
label_col = 'classification'
n_samples = 1_000_000   # REDUCED: was 2M (saves ~1.5GB per model)
n_models = 3            # how many separate models to train
random_seeds = [42, 101, 202]  # reproducible seeds

results = []

# --- Multi-sample training loop ---
for i, seed in enumerate(random_seeds[:n_models], start=1):
    print(f"\n🧩 Training model {i}/{n_models} on a random {n_samples:,} points...")

    # Sample randomly from the full dataset
    sample_df = df.sample(n=n_samples, random_state=seed)

    # Split
    X = sample_df[features].values
    y = sample_df[label_col].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, stratify=y, random_state=seed
    )

    # Train (reuses your function) - REDUCED: 100 estimators instead of 200
    clf, classes, report, cm = train_and_eval(X_train, X_test, y_train, y_test, n_estimators=100)

    # Collect results
    results.append({
        "seed": seed,
        "model": clf,
        "classes": classes,
        "report": report,
        "confusion": cm
    })

    # CLEANUP: Delete intermediate arrays and force garbage collection
    del sample_df, X, y, X_train, X_test, y_train, y_test
    gc.collect()

    print(f"✅ Finished model {i}/{n_models}")
    print(pd.DataFrame(report).T)
    plot_confusion(cm, classes, title=f"Confusion matrix (sample {i})", normalize=True)


In [ ]:
# we can also see all of the models' performance together by averaging their reports
all_reports = pd.concat([
    pd.DataFrame(r["report"]).T.assign(seed=r["seed"]) for r in results
])
display(all_reports)

> Sidenote: The thing I've found based on the exploration here is that when X and Y are removed from the equation, there is a DRASTIC reduction in accuracy. The model was hugely leveraging space as a predictive variable. Whilst we can keep this in, it makes it non-transferrable to other locations (unless we normalise and remove the exact geospatial co-ordinate, making each point relative to the points next to it).

> Better yet, we can use the calculated features, such as normals and slope, that Hendrik calculated in the larger pointcloud. Pro is this could improve accuracy for a single model for use in all pointclouds. Trade off is this file is absolutely huge. Still... We can implement all features and trim them off as and when...

## Experiment B: Using a 3 class system (Sidewalk, road, other)

In [ ]:
sidewalk_set = {2}
carriageway_set = {11}

def collapse_label(orig):
    if orig in sidewalk_set:
        return "sidewalk"
    elif orig in carriageway_set:
        return "carriageway"
    else:
        return "other"

In [ ]:
df["label_3class"] = df["classification"].apply(collapse_label)

In [ ]:
df["label_3class"].value_counts()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import gc

# --- Config ---
features = ['X', 'Y', 'Z', 'intensity', 'red', 'green', 'blue']
label_col = 'label_3class'
n_samples = 1_000_000   # REDUCED: was 2M (saves ~1.5GB per model)
n_models = 3            # how many separate models to train
random_seeds = [42, 101, 202]  # reproducible seeds

results_3class = []  # Use different variable name to avoid memory issues

# --- Multi-sample training loop ---
for i, seed in enumerate(random_seeds[:n_models], start=1):
    print(f"\n🧩 Training model {i}/{n_models} on a random {n_samples:,} points...")

    # Sample randomly from the full dataset
    sample_df = df.sample(n=n_samples, random_state=seed)

    # Split
    X = sample_df[features].values
    y = sample_df[label_col].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, stratify=y, random_state=seed
    )

    # Train (reuses your function) - REDUCED: 100 estimators instead of 200
    clf, classes, report, cm = train_and_eval(X_train, X_test, y_train, y_test, n_estimators=100)

    # Collect results
    results_3class.append({
        "seed": seed,
        "model": clf,
        "classes": classes,
        "report": report,
        "confusion": cm
    })

    # CLEANUP: Delete intermediate arrays and force garbage collection
    del sample_df, X, y, X_train, X_test, y_train, y_test
    gc.collect()

    print(f"✅ Finished model {i}/{n_models}")
    print(pd.DataFrame(report).T)
    plot_confusion(cm, classes, title=f"Confusion matrix (sample {i})", normalize=True)


## Experiment C: Spatial Validation with Tile-based Split

This experiment tests **transfer learning** by holding out complete spatial tiles from the training set. This simulates deploying the model to a new area of the same city, which is more realistic than random train/test splitting.

**Key question:** Does the model trained on one region work on an unseen spatial region?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# --- Create spatial tiles (grid-based) ---
# Divide the bounding box into tiles
n_tiles_x = 4  # Create a 4x4 grid
n_tiles_y = 4

x_min, x_max = df['X'].min(), df['X'].max()
y_min, y_max = df['Y'].min(), df['Y'].max()

print(f"Bounding box: X=[{x_min:.1f}, {x_max:.1f}], Y=[{y_min:.1f}, {y_max:.1f}]")

# Create tile assignment using numpy arrays (avoids memory issues)
x_linspace = np.linspace(x_min, x_max, n_tiles_x + 1)
y_linspace = np.linspace(y_min, y_max, n_tiles_y + 1)

tile_x = np.digitize(df['X'].values, x_linspace) - 1
tile_y = np.digitize(df['Y'].values, y_linspace) - 1

# Clip to valid range to handle edge cases
tile_x = np.clip(tile_x, 0, n_tiles_x - 1)
tile_y = np.clip(tile_y, 0, n_tiles_y - 1)

# Create tile IDs as numpy array (efficient)
tile_ids = np.array([f"{tx}_{ty}" for tx, ty in zip(tile_x, tile_y)])

# Count distribution
unique_tiles, counts = np.unique(tile_ids, return_counts=True)
print(f"\n✅ Created {len(unique_tiles)} spatial tiles ({n_tiles_x}x{n_tiles_y} grid)")
print(f"Tile distribution:")
for tile, count in sorted(zip(unique_tiles, counts)):
    print(f"  {tile}: {count:,} points")

# CLEANUP: Delete large tile assignment arrays
del tile_x, tile_y


In [ ]:
# --- Train/test split by spatial tiles ---
# Hold out one tile for testing, train on the rest
features = ['X', 'Y', 'Z', 'intensity', 'red', 'green', 'blue']
label_col = 'classification'

# Select a tile to hold out (e.g., tile 2_2 - center)
test_tile = '2_2'
train_mask = tile_ids != test_tile
test_mask = tile_ids == test_tile

# MEMORY OPTIMIZATION: Use numpy indexing to convert directly to arrays
# This avoids creating dataframe copies
train_indices = np.where(train_mask)[0]
test_indices = np.where(test_mask)[0]

X_train_spatial = df.iloc[train_indices][features].values
y_train_spatial = df.iloc[train_indices][label_col].values

X_test_spatial = df.iloc[test_indices][features].values
y_test_spatial = df.iloc[test_indices][label_col].values

print(f"📍 Spatial Split Configuration:")
print(f"   Training tiles: all except {test_tile}")
print(f"   Training points: {len(train_indices):,}")
print(f"   Test tile: {test_tile}")
print(f"   Test points: {len(test_indices):,}")
print(f"   Test data ratio: {len(test_indices) / len(df) * 100:.1f}%")

print(f"\n✅ Spatial split complete (using numpy arrays to save memory)")
print(f"   Training array shape: {X_train_spatial.shape}")
print(f"   Test array shape: {X_test_spatial.shape}")

# CLEANUP: Delete index arrays and masks
del train_indices, test_indices, train_mask, test_mask


In [ ]:
import gc

# --- Clean up large dataframes to free memory ---
print("🧹 Cleaning up large dataframes...", flush=True)
print(f"   Deleted: df, tile_ids", flush=True)

del df, tile_ids
gc.collect()

print(f"✅ Memory freed - numpy arrays retained for training", flush=True)
print(f"   Training array shape: {X_train_spatial.shape}", flush=True)
print(f"   Test array shape: {X_test_spatial.shape}", flush=True)


### this cell takes a LONG time without HPC

In [ ]:
# --- Train model on spatial split ---
import time

print("🧩 Training model on spatial split (all surrounding tiles)...\n")
print(f"   Training set size: {len(X_train_spatial):,} points")
print(f"   Test set size: {len(X_test_spatial):,} points")
print(f"   Features: {len(features)}")
print(f"   Estimators: 200")
print("\n⏳ This may take 5-15 minutes depending on system...\n")

start_time = time.time()

clf_spatial, classes_spatial, report_spatial, cm_spatial = train_and_eval(
    X_train_spatial, X_test_spatial, y_train_spatial, y_test_spatial,
    class_weight="balanced", n_estimators=200
)

elapsed = time.time() - start_time
print(f"\n✅ Training complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
print("\n📊 Performance on held-out spatial tile:")
print(pd.DataFrame(report_spatial).T)
plot_confusion(cm_spatial, classes_spatial, 
               title=f"Spatial Validation: Test on tile {test_tile}", normalize=True)

### Run this cell instead

In [ ]:
# --- FASTER VERSION: Train on sampled data ---
# This is much faster while still being statistically valid
import time
import sys

print("🧩 FASTER: Training model on SAMPLED spatial split...\n", flush=True)

# Sample training data for faster training (still statistically sound)
print("   Sampling training data...", flush=True)
sample_size = 1_000_000  # Use 1M points instead of all
sample_fraction = min(1.0, sample_size / len(X_train_spatial))
train_mask = np.random.RandomState(42).rand(len(X_train_spatial)) < sample_fraction
X_train_sample = X_train_spatial[train_mask]
y_train_sample = y_train_spatial[train_mask]

print(f"   Sampled training set: {len(X_train_sample):,} points (from {len(X_train_spatial):,})", flush=True)
print(f"   Full test set: {len(X_test_spatial):,} points", flush=True)
print(f"   Features: {len(features)}", flush=True)
print(f"   Estimators: 100 (reduced for speed)", flush=True)
print("\n⏳ This should take 2-5 minutes...\n", flush=True)

start_time = time.time()

clf_spatial_fast, classes_spatial_fast, report_spatial_fast, cm_spatial_fast = train_and_eval(
    X_train_sample, X_test_spatial, y_train_sample, y_test_spatial,
    class_weight="balanced", n_estimators=100
)

elapsed = time.time() - start_time
print(f"\n✅ Training complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)", flush=True)
print("\n📊 Performance on held-out spatial tile:", flush=True)
print(pd.DataFrame(report_spatial_fast).T)
plot_confusion(cm_spatial_fast, classes_spatial_fast, 
               title=f"Spatial Validation (FAST): Test on tile {test_tile}", normalize=True)

# Store for comparison
report_spatial = report_spatial_fast
cm_spatial = cm_spatial_fast

In [ ]:
results[0]

In [ ]:
report_spatial

In [ ]:
class_name

In [ ]:
# --- Compare: Random vs Spatial Split ---
print("=" * 70)
print("COMPARISON: Random Split vs Spatial Split")
print("=" * 70)

# Extract accuracy metrics from both experiments
random_split_results = results[0]  # From Experiment A (all classes)
spatial_split_report = report_spatial

# Compare accuracy for each class
print("\n🎯 Overall Accuracy:")
print(f"   Random split (Exp A):  {random_split_results['report']['accuracy']:.4f}")
print(f"   Spatial split (Exp C): {spatial_split_report['accuracy']:.4f}")

print("\n📈 Per-class F1-scores:")
# Iterate over keys in the report that are class labels (stored as strings)
for key in random_split_results['report'].keys():
    # Skip non-numeric keys like 'accuracy', 'macro avg', 'weighted avg', etc.
    try:
        class_label = int(key)  # Try to convert string key to int
        # Now check both the string key and int key in the spatial report
        if key in spatial_split_report:
            random_f1 = random_split_results['report'][key]['f1-score']
            spatial_f1 = spatial_split_report[key]['f1-score']
            diff = spatial_f1 - random_f1
            symbol = "📉" if diff < -0.05 else "📈" if diff > 0.05 else "➡️"
            print(f"   Class {class_label:2} | Random: {random_f1:.3f} | Spatial: {spatial_f1:.3f} | Δ {diff:+.3f} {symbol}")
    except (ValueError, KeyError):
        # Skip keys that can't be converted to int or aren't in spatial report
        pass

print("\n💡 Interpretation:")
print("   • If spatial performance is similar → model generalizes well to unseen regions")
print("   • If spatial performance drops significantly → model is overfitting to local patterns")
print("   • Spatial validation is more realistic for deployment to new areas")


### This cell checks for tile contiguity and computes mean tile parameters for each class

**Motivation:** We've found that random selection of tiles for training the model actually moderately increases accuracy. Now this could be because we´re currently just taking a sample of 1m and may change if we're using all of them. This is why we're going to take the radioetric properties of each contiguous cell, because if we have a random sample across the pointcloud, we might be creating a more general model, rather than a localised one.

In [ ]:
# --- Diagnostics: tiles, radiometrics, SHAP analysis, and plotting helpers ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

# Assumptions:
# - `X_train_spatial`, `X_test_spatial` are numpy arrays with columns [X, Y, Z, intensity, red, green, blue]
# - `report_spatial` and `results` exist from earlier cells
# - `clf` (random model) and `clf_spatial_fast` exist if you ran both training cells

feature_names = ['X', 'Y', 'Z', 'intensity', 'red', 'green', 'blue']

# 1) Reconstruct tile grid from combined spatial arrays (safe if `df` was deleted)
all_XY = np.vstack([X_train_spatial[:, :2], X_test_spatial[:, :2]])

n_tiles_x = globals().get('n_tiles_x', 4)
n_tiles_y = globals().get('n_tiles_y', 4)

x_min, x_max = all_XY[:,0].min(), all_XY[:,0].max()
y_min, y_max = all_XY[:,1].min(), all_XY[:,1].max()

x_linspace = np.linspace(x_min, x_max, n_tiles_x + 1)
y_linspace = np.linspace(y_min, y_max, n_tiles_y + 1)

# helper to compute tile ids for an XY array
def compute_tile_ids(xy):
    tx = np.digitize(xy[:,0], x_linspace) - 1
    ty = np.digitize(xy[:,1], y_linspace) - 1
    tx = np.clip(tx, 0, n_tiles_x - 1)
    ty = np.clip(ty, 0, n_tiles_y - 1)
    return np.array([f"{a}_{b}" for a,b in zip(tx, ty)])

# Compute tile_ids for train and test arrays
tile_ids_train = compute_tile_ids(X_train_spatial[:, :2])
tile_ids_test = compute_tile_ids(X_test_spatial[:, :2])

# Quick contiguity check: plot a small sample colored by tile id
def plot_tile_samples(sample_size=200000):
    import matplotlib.pyplot as plt
    # sample from combined XY
    total = len(tile_ids_train) + len(tile_ids_test)
    sample_size = min(sample_size, total)
    # sample proportionally from train and test
    n_train = int(sample_size * (len(tile_ids_train)/total))
    n_test = sample_size - n_train

    idx_train = np.random.RandomState(0).choice(len(tile_ids_train), n_train, replace=False) if n_train>0 else np.array([], dtype=int)
    idx_test = np.random.RandomState(1).choice(len(tile_ids_test), n_test, replace=False) if n_test>0 else np.array([], dtype=int)

    XY_sample = np.vstack([X_train_spatial[idx_train, :2] if n_train>0 else np.zeros((0,2)),
                           X_test_spatial[idx_test, :2] if n_test>0 else np.zeros((0,2))])
    tile_sample = np.concatenate([tile_ids_train[idx_train] if n_train>0 else np.array([], dtype=object),
                                  tile_ids_test[idx_test] if n_test>0 else np.array([], dtype=object)])

    # map tile ids to integer colors
    unique_tiles = np.unique(tile_sample)
    tile_to_int = {t:i for i,t in enumerate(unique_tiles)}
    colors = np.array([tile_to_int[t] for t in tile_sample])

    plt.figure(figsize=(8,6))
    sc = plt.scatter(XY_sample[:,0], XY_sample[:,1], c=colors, s=1, cmap='tab20', alpha=0.8)
    plt.title('Sampled points colored by spatial tile')
    plt.xlabel('X'); plt.ylabel('Y')
    plt.tight_layout()
    plt.show()

print('🧭 Running quick spatial contiguity plot (may sample up to 200k points)...')
plot_tile_samples(sample_size=100000)

# 2) Per-tile radiometrics: compute mean intensity and colors for each tile (train + test separately)

def per_tile_radiometrics(X_arr, tile_ids_arr):
    df_r = pd.DataFrame({
        'tile': tile_ids_arr,
        'intensity': X_arr[:,3].astype(float),
        'red': X_arr[:,4].astype(float),
        'green': X_arr[:,5].astype(float),
        'blue': X_arr[:,6].astype(float)
    })
    grouped = df_r.groupby('tile').mean()
    return grouped.sort_index()

print('\n📊 Computing per-tile radiometrics (train)...')
train_radiom = per_tile_radiometrics(X_train_spatial, tile_ids_train)
print(train_radiom)

print('\n📊 Computing per-tile radiometrics (test)...')
test_radiom = per_tile_radiometrics(X_test_spatial, tile_ids_test)
print(test_radiom)

# Plot intensity differences across tiles
plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
train_radiom['intensity'].plot(kind='bar'); plt.title('Train: mean intensity per tile')
plt.subplot(1,2,2)
test_radiom['intensity'].plot(kind='bar'); plt.title('Test: mean intensity per tile')
plt.tight_layout(); plt.show()

# 3) SHAP analysis helpers (trees only). Uses small sample to limit RAM.
print('\n🧠 Preparing SHAP analysis helpers (will sample up to 5000 rows).')

def run_shap_for_model(model, X_array, feature_names, sample_size=5000):
    try:
        import shap
    except Exception as e:
        print('✖ shap not installed or failed to import. Install with `pip install shap` to run SHAP analysis.')
        return None

    n = min(sample_size, len(X_array))
    idx = np.random.RandomState(0).choice(len(X_array), n, replace=False)
    X_sample = X_array[idx]

    # Use TreeExplainer for tree-based models
    try:
        explainer = shap.TreeExplainer(model)
    except Exception as e:
        print('✖ TreeExplainer failed:', e)
        return None

    shap_values = explainer.shap_values(X_sample)

    # shap_values for multiclass is a list; convert to mean absolute across classes
    if isinstance(shap_values, list):
        # compute mean absolute shap across classes for each feature
        abs_mean = np.mean([np.abs(sv).mean(axis=0) for sv in shap_values], axis=0)
        feature_imp = pd.Series(abs_mean, index=feature_names).sort_values(ascending=False)
    else:
        feature_imp = pd.Series(np.abs(shap_values).mean(axis=0), index=feature_names).sort_values(ascending=False)

    # Summary plot (may be slow) — show if interactive
    try:
        shap.summary_plot(shap_values, X_sample, feature_names=feature_names, show=True)
    except Exception:
        pass

    return feature_imp

# Run SHAP for random model (if available)
if 'clf' in globals():
    print('\n🔎 SHAP for random (global) model — this may take a minute')
    imp_random = run_shap_for_model(clf, np.vstack([X_train_spatial, X_test_spatial]), feature_names, sample_size=3000)
    if imp_random is not None:
        print('\nFeature importance (random model):')
        print(imp_random)

if 'clf_spatial_fast' in globals():
    print('\n🔎 SHAP for spatial model (fast) — this may take a minute')
    imp_spatial = run_shap_for_model(clf_spatial_fast, np.vstack([X_train_spatial, X_test_spatial]), feature_names, sample_size=3000)
    if imp_spatial is not None:
        print('\nFeature importance (spatial model):')
        print(imp_spatial)

# 4) Tile plotting helpers: 2D scatter and optional Open3D 3D viewer

def plot_tile_2d(tile_id, which='train', sample=50000):
    if which == 'train':
        Xarr, tids = X_train_spatial, tile_ids_train
    else:
        Xarr, tids = X_test_spatial, tile_ids_test
    mask = (tids == tile_id)
    if mask.sum() == 0:
        print(f'No points for tile {tile_id} in {which}')
        return
    idx = np.where(mask)[0]
    n = min(sample, len(idx))
    sel = np.random.RandomState(0).choice(idx, n, replace=False)
    XY = Xarr[sel, :2]
    plt.figure(figsize=(6,5))
    plt.scatter(XY[:,0], XY[:,1], s=1)
    plt.title(f'Tile {tile_id} ({which}) — {n} points')
    plt.xlabel('X'); plt.ylabel('Y')
    plt.show()

def plot_tile_3d_open3d(tile_id, which='train', sample=100000):
    try:
        import open3d as o3d
    except Exception as e:
        print('Open3D not available.')
        return
    if which == 'train':
        Xarr, tids = X_train_spatial, tile_ids_train
    else:
        Xarr, tids = X_test_spatial, tile_ids_test
    mask = (tids == tile_id)
    if mask.sum() == 0:
        print(f'No points for tile {tile_id} in {which}')
        return
    idx = np.where(mask)[0]
    n = min(sample, len(idx))
    sel = np.random.RandomState(0).choice(idx, n, replace=False)
    pts = Xarr[sel, :3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    o3d.visualization.draw_geometries([pcd], window_name=f'Tile {tile_id} ({which})')

print('\n✅ Diagnostics and helpers added. Examples:')
print("  - plot_tile_samples(sample_size=50000)")
print("  - plot_tile_2d('1_1', which='train')")
print("  - plot_tile_3d_open3d('2_2', which='test')")
print("  - run_shap_for_model(clf, np.vstack([X_train_spatial, X_test_spatial])[:,3:], ['intensity','red','green','blue'])  # model on radiometric features only")

gc.collect()


## Interactive viewing of model labels (predicted and True)

In [ ]:
import open3d as o3d
import numpy as np
import gc

# MEMORY NOTE: This cell uses X_test_spatial which is already loaded
# Create a visualization sample to avoid loading too much data
print("Creating visualization sample (100k points)...", flush=True)

# Sample indices from the test set 
sample_indices = np.random.RandomState(42).choice(len(X_test_spatial), 
                                                   min(100_000, len(X_test_spatial)), 
                                                   replace=False)
X_plot = X_test_spatial[sample_indices]

# get predicted labels from the trained model
y_pred = clf_spatial_fast.predict(X_plot)

# convert numeric codes to strings
class_map = {
    1: "carriageway",
    2: "sidewalk",
    3: "other",
    11: "carriageway",
    "carriageway": "carriageway",
    "sidewalk": "sidewalk",
    "other": "other"
}
labels = pd.Series(y_pred).map(class_map).fillna("other")

# assign RGB colors per predicted class
color_map = {
    "carriageway": [0.6, 0.6, 0.6],  # grey
    "sidewalk": [1.0, 0.5, 0.1],     # orange
    "other": [0.3, 0.8, 0.3]         # green
}
colors = np.array([color_map[lbl] for lbl in labels])

# create point cloud for visualization
# Extract X, Y, Z from the feature array
pcd_pred = o3d.geometry.PointCloud()
pcd_pred.points = o3d.utility.Vector3dVector(X_plot[:, :3])  # First 3 columns are X, Y, Z
pcd_pred.colors = o3d.utility.Vector3dVector(colors)

# visualize
o3d.visualization.draw_geometries([pcd_pred], window_name="Predicted Labels")

# CLEANUP: Free memory after visualization
del X_plot, y_pred, colors, labels, pcd_pred, sample_indices
gc.collect()
print("✅ Visualization complete and memory cleaned up", flush=True)
